# Análisis de datos de Radiodifusión en AM/FM

## Unidad de Medios y Contenidos Audiovisuales
### Subdirección de Validación
---

<img src="./Datos/assets/IFTheader.png"
     alt="IFTicon"
     style = 'position:absolute; top:0; right:0; padding:2px;'/>
     
     
 

La presente *notebook*, tiene por objetivo ser una herramienta de análisis y trabajo, que de la mano con las bases de datos, faciliten y optimicen los procesos realizados dentro de la subdirección. Especificamente, aquellos que implican el procesamiento de coberturas, listado de localidades, valores porcentuales de cobertura y extensión geográfica; por ejemplo, las publicaciones del análisis para la **Retransmisión de señales**, cuyo sustento es el análisis de cobertura porcentual nácional por cada canal de programación transmitido por las estaciones de TDT.

En primera instancia, se realiza a manera de ejemplo, el análisis para las estaciones de radiodifusión sonora en AM y FM. Este primer ejemplo, explicará los requerimientos de procesamiento, formato de datos, bibliotecas empleadas y demás pormenores que deberán ser atendidos para obtener resultados consistentes al emplear esta herramienta.

Una de las ventajas y un objetivo adicional que se busca con la integración de la presente *notebook* es facilitar la lectura, mantenimiento y entendimiento del código que se utiliza en los análisis que ya se realizan al día de hoy mediante una adecuada documentación.

# Radiodifusión sonora en FM


Importamos las bibliotecas necesarias para el procesamiento y análisis de datos; así como la visualización y tratamiento de datos georeferenciados.

In [1]:
import csv #For .csv format files
import glob #Helps to use unix-like path expresions, (/ , ./ , ..) 
import os #Write and read data from filesystem.
import pandas as pd 
import collections
import geopandas as gpd
from tqdm.notebook import tqdm, trange

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import mapclassify as mc
import matplotlib as mpl

# from tqdm import trange, tqdm


In [2]:
from IPython.display import display, Markdown

In [3]:
!pip install jupyter_bokeh 

In [4]:
# !pip install mapclassify
# !pip install openpyxl
# !pip install geopandas
# from mpl_toolkits.basemap import Basemap

In [5]:
#Definimos la ruta del archivo base de localidades,con la cual se compararan las coberturas.

inegi_path='./Datos/Localidades_20100.csv'
inegi_data = pd.read_csv(inegi_path,header=0, delimiter = ";", encoding = 'latin-1')
# print (inegi_data)
# print (inegi_data.columns)

distintivos= []
localidades=[]
nom_loc = []
history = []


Podemos observar la estrucutra de la tabla base de localidades INEGI 2010:

In [6]:
inegi_data
pob_total_nac = inegi_data['POB_TOTAL'].sum(axis=0)

In [7]:
inegi_data

,_ID,NO_ENT,ENTIDAD,NO_MPIO,MUNICIPIO,NO_LOC,Location,GM_2010,LONG,LAT,...,MUJERES,TOTAL_HOGA,VIVPARHA,O_VIVPAR,VPH_RADIO,VPH_TV,POB_0_17,POB_18_59,POB_MAYOR_,OID
0,10010001,1,Aguascalientes,1,Aguascalientes,1,AGUASCALIENTES,Muy bajo,1021746,215251,...,373528.0,184123.0,185050.0,718066.0,167880.0,181062.0,255352.0,410259.0,56639.0,1
1,10010094,1,Aguascalientes,1,Aguascalientes,94,GRANJA ADELITA,x,1022225,215219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,10010096,1,Aguascalientes,1,Aguascalientes,96,AGUA AZUL,Alto,1022127,215303,...,16.0,11.0,11.0,37.0,8.0,11.0,12.0,21.0,4.0,3
3,10010100,1,Aguascalientes,1,Aguascalientes,100,RANCHO ALEGRE,x,1022222,215116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,10010102,1,Aguascalientes,1,Aguascalientes,102,LOS ARBOLITOS [RANCHO],x,1022126,214649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192240,320580036,32,Zacatecas,58,Santa María de la Paz,36,MARINES,x,1032337,213347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192241
192241,320580037,32,Zacatecas,58,Santa María de la Paz,37,MESA GRANDE,Medio,1032457,213317,...,99.0,47.0,47.0,192.0,43.0,43.0,71.0,38.0,83.0,192242
192242,320580039,32,Zacatecas,58,Santa María de la Paz,39,SAN ISIDRO,x,1032014,212916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192243
192243,320580041,32,Zacatecas,58,Santa María de la Paz,41,SAN MIGUEL TEPETITLÁN,Medio,1032007,213014,...,34.0,29.0,29.0,75.0,24.0,25.0,14.0,26.0,35.0,192244


In [8]:
pob_total_nac

112336538

In [9]:
mycsvdir = './Datos/FM/' #Local path, you can change it to mkae it matc your files route. i.e. '/csv/' , '../../Documents/"
FMcsvfiles = glob.glob(os.path.join(mycsvdir, '*.csv')) #join local path with all .csv files in that route/folder; then group all filenames&file routes in an array called csvfiles.
# print (csvfiles) #Shows csvfiles array. Can be removed.
ides =[]

Se leen los documentos .csv que se encuentren en la ruta específicada; en este caso, los archivos csv de la cobertura de las estaciones de FM.

In [10]:
i=0
for csvfile in tqdm(FMcsvfiles):
    rawdata=pd.read_csv(csvfile,header=2, encoding = 'latin-1')
    rawdata.drop(rawdata.tail(3).index, inplace=True) #Sacamos las ultimas 3 filas del dataset
    # rawdata = rawdata.drop(rawdata[rawdata['###']==True].index)
    for i in rawdata.index:
        ides.append(rawdata["_ID"][i])
#     print ("leido" + csvfile )
Incidenciasperstation = collections.Counter(ides)
# print (Incidenciasperstation)

  0%|          | 0/1393 [00:00<?, ?it/s]

In [11]:
FM_pdarray = pd.DataFrame(list(Incidenciasperstation.items()),columns = ['_ID','Num']) 
# print (pd_array)
FM_pdarray['_ID']=FM_pdarray['_ID'].astype('int64')

tqdm.pandas()
FM_pdarray = pd.merge(FM_pdarray,inegi_data[['_ID','ENTIDAD','NO_ENT','LONG','LAT','LONGITUD','LATITUD','POB_TOTAL','VPH_RADIO']], on='_ID', how='left').progress_apply(lambda x: x)
# print (pd_array)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
print ('Por favor espera 1/3 ...')
writer = pd.ExcelWriter((r'./Datos/listaRadioFM3.xlsx'),engine='openpyxl')
print ('Por favor espera 2/3 ...')
FM_pdarray.to_excel(writer,index = True, header=True)
print ('Por favor espera 3/3 ...')
writer.save()
print ('Listo')

Por favor espera 1/3 ...
Por favor espera 2/3 ...


In [ ]:
FM_pdarray

Ahora se ha generado un Dataframe que contiene para cada una las localidades dentro de la cobertura general :

1. ID de la localidad.
2. Número de estaciones de **Radio en FM** que la contienen dentro de su cobertura.
3. Nombre de la Entidad a la que pertenece.
4. Coordenadas geofráficas (Hexagesimales y Decimales en grados).
5. Población total.
6. Viviendas particulares habitadas que cuentan con radio.

Es posible generar datos con valor agregado para cualquier análisis que nos interese. En este caso, generaremos un mapa de localidades con cobertura de estaciones de **Radio en FM**, que distinga el número de estaciones que podrían ser sintonizadas en cada una de ellas, su impacto a nivel de población e intentaremos encontrar una correlación o tendencia que relacione dichos datos gráficos. 
Esta información, a nivel de *negocio*, permite tomar decisiones interesantes, o encontrar relaciones que sustenten dichas decisiones, por ejemplo, una relación directa entre los centros urbanos y la concentración de estaciones de radio FM, el alcance de las coberturas de las estaciones; y a nivel estratégico, la localización de infraestructura de radioodifusión, que puede estar asociada con la presencia instalaciones de energía eléctrica de alta potencia por ejemplo.

In [ ]:
FMdata = pd.Series(FM_pdarray["Num"])
cmap = plt.get_cmap('viridis', 15)
# print (list(pd_array["Num"]))
bins_NB = mc.NaturalBreaks(FMdata,15).bins
norm = mpl.colors.BoundaryNorm(bins_NB, cmap.N)

In [ ]:
fig, ax_FM = plt.subplots(figsize=(22, 14))
ax_FM.set_aspect('equal')
fp = './Datos/geodata/estatal.shp'

base0 = gpd.read_file(fp, encoding = "utf-8")
base0.crs = "EPSG:6362"
# base = base0.to_crs({'init': 'epsg:4326'})
base = base0.to_crs("EPSG:4326")
base.plot(ax=ax_FM, color='white', edgecolor='black')

# basee = ax.plot(alpha=0.5, edgecolor='k',color='white')

puntos = ax_FM.scatter(
    FM_pdarray["LONGITUD"],
    FM_pdarray["LATITUD"],
    s=0.2,
    alpha=1,
    c=FM_pdarray["Num"],
    cmap=cmap,
    norm=norm,
)

ax_FM.set_title("Estaciones de Radio FM en México")
ax_FM.set_xlabel("Longitud")
ax_FM.set_ylabel("Latitud")
cb = fig.colorbar(puntos)
cb.set_label("Número de estaciones")

---
# Radiodifusión sonora en AM

Dada la nomenclatura de los archivos de AM; es necesario agregar un diccionario que determine la localidad a la que pertenece cada estación.

In [ ]:
dictloc = {'ags':'Aguascalientes', 'bc':'Baja California', 'bcs':'Baja California Sur', 'camp':'Campeche', 'coah':'Coahuila de Zaragoza', 'col':'Colima', 'chis':'Chiapas', 'chih':'Chihuahua', 'cdmx':'Ciudad de México', 'dgo':'Durango', 'gto':'Guanajuato','gro':'Guerrero', 'hgo':'Hidalgo', 'jal':'Jalisco','mex':'México', 'mich':'Michoacán de Ocampo', 'mor':'Morelos', 'nay':'Nayarit', 'nl':'Nuevo León','oax':'Oaxaca','pue':'Puebla', 'qro':'Querétaro', 'qroo':'Quintana Roo', 'slp':'San Luis Potosí', 'sin':'Sinaloa', 'son':'Sonora','tab':'Tabasco','tams':'Tamaulipas','tamps':'Tamaulipas','tlax':'Tlaxcala','ver':'Veracruz de Ignacio de la Llave', 'yuc':'Yucatan', 'zac':'Zacatecas'}
AMcsvdir = './Datos/AM/' #Local path, you can change it to mkae it matc your files route. i.e. '/csv/' , '../../Documents/"
AMcsvfiles = glob.glob(os.path.join(AMcsvdir, '*.csv')) #join local path with all .csv files in that route/folder; then group all filenames&file routes in an array called csvfiles.
# print (csvfiles) #Shows csvfiles array. Can be removed.
AMides =[]

In [ ]:
i=0
for AMcsvfile in tqdm(AMcsvfiles):
    rawdata=pd.read_csv(AMcsvfile,encoding = 'latin-1')
#     rawdata.drop(rawdata.tail(3).index, inplace=True) #Sacamos las ultimas 3 filas del dataset
    # rawdata = rawdata.drop(rawdata[rawdata['###']==True].index)
    for i in rawdata.index:
        AMides.append(rawdata["_ID"][i])
#     print ("leido" + csvfile )
Incidenciasperstation = collections.Counter(AMides)

# print (Incidenciasperstation)

In [ ]:
AM_pdarray =[]
AM_pdarray = pd.DataFrame(list(Incidenciasperstation.items()),columns = ['_ID','Num']) 

AM_pdarray = AM_pdarray.apply(pd.to_numeric, errors='coerce')
AM_pdarray.dropna(inplace= True)

# print (AM_pdarray)
tqdm.pandas()
AM_pdarray = pd.merge(AM_pdarray,inegi_data[['_ID','ENTIDAD','NO_ENT','LONG','LAT','LONGITUD','LATITUD','POB_TOTAL','VPH_RADIO']], on='_ID', how='left').progress_apply(lambda x: x)
# print (AM_pdarray)

In [ ]:
print ('Por favor espera 1/3 ...')
writerAM = pd.ExcelWriter((r'./Datos/CoberturaGralRadioAM.xlsx'),engine='openpyxl')
print ('Por favor espera 2/3 ...')
AM_pdarray.to_excel(writerAM,index = True, header=True)
print ('Por favor espera 3/3 ...')
writer.save()
print ('ready')

In [ ]:
AMdata = pd.Series(AM_pdarray["Num"])
cmap = plt.get_cmap('viridis', 15)
# print (list(pd_array["Num"]))
bins_NB_AM = mc.NaturalBreaks(AMdata,15).bins
norm = mpl.colors.BoundaryNorm(bins_NB_AM, cmap.N)

In [ ]:
# AMdata

In [ ]:
fig_AM, ax_AM = plt.subplots(figsize=(22, 14))
ax_AM.set_aspect('equal')

base.plot(ax=ax_AM, color='white', edgecolor='black')

puntos_AM = ax_AM.scatter(
    AM_pdarray["LONGITUD"],
    AM_pdarray["LATITUD"],
    s=0.2,
    alpha=1,
    c=AM_pdarray["Num"],
    cmap=cmap,
    norm=norm,
)

ax_AM.set_title("Estaciones de Radio AM en México")
ax_AM.set_xlabel("Longitud")
ax_AM.set_ylabel("Latitud")
cb = fig_AM.colorbar(puntos_AM)
cb.set_label("Número de estaciones")


In [ ]:
diff = pd.merge(FM_pdarray[['_ID']], inegi_data[['_ID','ENTIDAD','NO_ENT','LONG','LAT','LONGITUD','LATITUD','POB_TOTAL']],on='_ID', how = 'outer', indicator= True).loc[lambda x : x['_merge']=='right_only']

Es posible determinar cuales localidades no se encuentran dentro de la cobertura general, y reflejar dicho listado en un mapa similar al de la cobertura, pero en este caso, para mostrar aquellas localidades que según los archivos de cobertura, no se encuentran en la posiblidad de sintonizar alguna estación de radio FM.

In [ ]:
diff

In [ ]:
pob_no_FM = diff['POB_TOTAL'].sum(axis=0)

In [ ]:
pob_no_FM

In [ ]:
fig3, ax3 = plt.subplots(figsize=(18, 13))
ax3.set_aspect('equal')
fp = './Datos/geodata/estatal.shp'

base.plot(ax=ax3, color='white', edgecolor='black')
puntos = ax3.scatter(
    diff["LONGITUD"],
    diff["LATITUD"],
    s=0.2,
    alpha=1,
    cmap=cmap,
    norm=norm,
)

ax3.set_title("Localidades sin acceso al señal de Radio FM en México")
ax3.set_xlabel("Longitud")
ax3.set_ylabel("Latitud")

In [ ]:
s = len(diff)
Markdown(f"**Número de localidades a nivel nacional que no cuentan con acceso a Radio FM**: {s}")


In [ ]:
diff_am = pd.merge(AM_pdarray[['_ID']], inegi_data[['_ID','ENTIDAD','NO_ENT','LONG','LAT','LONGITUD','LATITUD','POB_TOTAL']],on='_ID', how = 'outer', indicator= True).loc[lambda x : x['_merge']=='right_only']
pob_no_AM = diff_am['POB_TOTAL'].sum()

In [ ]:
diff_am

In [ ]:
pob_no_AM

In [ ]:
fig4, ax4 = plt.subplots(figsize=(18, 13))
ax4.set_aspect('equal')
fp = './Datos/geodata/estatal.shp'
base.plot(ax=ax4, color='white', edgecolor='black')

puntos = ax4.scatter(
    diff_am["LONGITUD"],
    diff_am["LATITUD"],
    s=0.2,
    alpha=1,
    cmap=cmap,
    norm=norm,
)

ax4.set_title("Localidades sin acceso a señal de Radio AM en México")
ax4.set_xlabel("Longitud")
ax4.set_ylabel("Latitud")

## Análisis conjunto de Radiodifusión sonora. 

In [ ]:
new_base = base.rename(columns={'CVEGEO':'NO_ENT'})

In [ ]:
state_locations_FM = FM_pdarray.groupby(['NO_ENT']).count()
state_locations_FM=state_locations_FM.reset_index()
state_locations_FM['NO_ENT']=state_locations_FM['NO_ENT'].astype('int64')

state_locations_AM = AM_pdarray.groupby(['NO_ENT']).count()
state_locations_AM=state_locations_AM.reset_index()
state_locations_AM['NO_ENT']=state_locations_AM['NO_ENT'].astype('int64')



general_locations = inegi_data.groupby(['NO_ENT']).count()
general_locations=general_locations.reset_index()
general_locations['NO_ENT']=general_locations['NO_ENT'].astype('int64')


In [ ]:
fig = plt.figure(figsize = (20,13))
fig.suptitle('Comparación de mapas de presencia de Radiodifusión en AM y FM')

gs = fig.add_gridspec(2, 2, hspace=0, wspace=0)
(ax1, ax2), (ax3, ax4) = gs.subplots(sharex='col', sharey='row')

# axs[1,1].plot(base)

base.plot(ax=ax1, color='white', edgecolor='black')
base.plot(ax=ax2, color='white', edgecolor='black')
base.plot(ax=ax3, color='white', edgecolor='black')
base.plot(ax=ax4, color='white', edgecolor='black')

puntos = ax1.scatter(
    AM_pdarray["LONGITUD"],
    AM_pdarray["LATITUD"],
    s=0.2,
    alpha=1,
    c=AM_pdarray["Num"],
    cmap=cmap,
    norm=norm,
)
ax1.set_title("Cobertura Radio AM")

puntos2 = ax2.scatter(
    FM_pdarray["LONGITUD"],
    FM_pdarray["LATITUD"],
    s=0.2,
    alpha=1,
    c=FM_pdarray["Num"],
    cmap=cmap,
    norm=norm,
)
ax2.set_title("Cobertura Radio FM")
puntos = ax4.scatter(
    diff["LONGITUD"],
    diff["LATITUD"],
    s=0.2,
    alpha=1,
    cmap=cmap,
    norm=norm,
)

puntos = ax3.scatter(
    diff_am["LONGITUD"],
    diff_am["LATITUD"],
    s=0.2,
    alpha=1,
    cmap=cmap,
    norm=norm,
)

## Summary and Results

In [ ]:
difloc = {}
percentajes = {}
for i in general_locations.index:
    difloc[i] = i+1, general_locations["_ID"][i] - state_locations_FM["_ID"][i], round((state_locations_FM["_ID"][i]/general_locations["_ID"][i])*100,3), round((state_locations_FM["_ID"][i]/general_locations["_ID"][i])*100,3).astype(str) + '%',general_locations["_ID"][i] - state_locations_AM["_ID"][i], round((state_locations_AM["_ID"][i]/general_locations["_ID"][i])*100,3), round((state_locations_AM["_ID"][i]/general_locations["_ID"][i])*100,3).astype(str) + '%'
    percentajes[i] = i+1, round((state_locations_FM["_ID"][i]/general_locations["_ID"][i])*100,3)

In [ ]:
difloc_pd = pd.DataFrame.from_dict(data = difloc, orient = 'index', columns = ['NO_ENT','Diferencia_FM','Percent_FM','Percent_print_FM','Diferencia_AM','Percent_AM','Percent_print_AM'])

In [ ]:
new_base
new_base['NO_ENT']=new_base['NO_ENT'].astype('int64')

In [ ]:
n_per_state = pd.merge(new_base,difloc_pd[['NO_ENT','Diferencia_FM','Percent_FM','Percent_print_FM','Diferencia_AM','Percent_AM','Percent_print_AM']], on='NO_ENT', how='left').progress_apply(lambda x: x)
# n_per_state = pd.merge(new_base,state_locations[['NO_ENT','Num']], on='NO_ENT', how='left').progress_apply(lambda x: x)

n_per_state = n_per_state[['NO_ENT','NOM_ENT','POBTOT','Diferencia_FM','Percent_FM','Percent_print_FM','Diferencia_AM','Percent_AM','Percent_print_AM','geometry']]

In [ ]:
import json
import jupyter_bokeh
import bokeh
from bokeh.io import show, output_notebook, push_notebook
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = n_per_state.to_json())

In [ ]:
# Create figure object.
output_notebook()
p = figure(title = 'Localidades con cobertura de Radio FM', 
           plot_height = 700 ,
           plot_width = 1000, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = 'moccasin',
                   line_color = 'gray', 
                   line_width = 2, 
                   fill_alpha = 0.3)
# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('Estado','@NOM_ENT'),
                                ('Población','@POBTOT'),
                                 ('Localidades no incluidas FM', '@Diferencia_FM'),
                                 ('Porcentaje de cobertura FM', '@Percent_print_FM'),
                                 ('Localidades no incluidas AM', '@Diferencia_AM'),
                                 ('Porcentaje de cobertura AM', '@Percent_print_AM')]))



# show(p)

In [ ]:
n_per_state = n_per_state.sort_values(by='Percent_AM', ascending=True)

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
from bokeh.transform import dodge


# p2 = figure(title = "Estados" )
# right = n_per_state['NOM_ENT']
# y = n_per_state['Percent']
# height = 0.5
# p2.hbar(y = 'right' ,right= y, height=height)


estados = n_per_state['NOM_ENT']
# porcentajes =  n_per_state['Percent']
source = ColumnDataSource(n_per_state.drop(columns=['geometry']))

p2 = figure(y_range=estados, x_range=(0, 100), plot_width=550, title="Cobertura de radiodifusión AM/FM", toolbar_location=None, tools="")

p2.hbar(y=dodge('NOM_ENT',  0.35,  range=p2.y_range), right='Percent_AM', height=0.3, source=source,
       color="#718dbf",legend_label='AM')

p2.hbar(y=dodge('NOM_ENT', 0, range=p2.y_range), right='Percent_FM', height=0.3, source=source,
       color="salmon",legend_label='FM')



p2.legend.location='bottom_right'

p2.y_range.range_padding = 0
p2.ygrid.grid_line_color = None

layout = bokeh.layouts.layout([
    [p,p2]

])


In [ ]:
show(layout)

In [ ]:
n_per_state[['NOM_ENT','POBTOT','Percent_print_AM','Percent_print_FM']]

In [ ]:
from math import pi
import pandas as pd
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.models import LabelSet, ColumnDataSource

x = {
    'Poblacion con Radio FM': pob_total_nac-pob_no_FM,
    'Población sin Radio FM': pob_no_FM
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['printable_value'] = f' {pob_total_nac-pob_no_FM:,}' , f' {pob_no_FM:,}'
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = '#718dbf','salmon'

pie_1 = figure(plot_height=350, title="Relación de población con Radio FM", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

pie_1.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

data["printable_value"] = data['printable_value'].astype(str)
data["printable_value"] = data["printable_value"].str.pad(47, side = "left")
source = ColumnDataSource(data)

labels = LabelSet(x=0, y=1,text=('printable_value'), angle=cumsum('angle', include_zero=True), source=source, render_mode='canvas')

pie_1.add_layout(labels)

pie_1.axis.axis_label=None
pie_1.axis.visible=False
pie_1.grid.grid_line_color = None



##################################

x2 = {
    'Poblacion con Radio AM': pob_total_nac-pob_no_AM,
    'Población sin Radio AM': pob_no_AM
}

data_AM = pd.Series(x2).reset_index(name='value').rename(columns={'index':'country'})
data_AM['printable_value'] = f' {pob_total_nac-pob_no_AM:,}' , f' {pob_no_AM:,}'
data_AM['angle'] = data_AM['value']/data_AM['value'].sum() * 2*pi
data_AM['color'] = '#718dbf','salmon'

pie_2 = figure(plot_height=350, title="Relación de población con Radio AM", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

pie_2.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data_AM)

data_AM["printable_value"] = data_AM['printable_value'].astype(str)
data_AM["printable_value"] = data_AM["printable_value"].str.pad(47, side = "left")
source = ColumnDataSource(data_AM)

labels = LabelSet(x=0, y=1,text=('printable_value'), angle=cumsum('angle', include_zero=True), source=source, render_mode='canvas')

pie_2.add_layout(labels)

pie_2.axis.axis_label=None
pie_2.axis.visible=False
pie_2.grid.grid_line_color = None


layout_pie = bokeh.layouts.layout([
    [pie_1,pie_2]])

show(layout_pie)

In [ ]:
per_no_fm = str(round (((pob_total_nac-pob_no_FM)/(pob_total_nac))*100,3)) + "%"

per_no_am =  (pob_total_nac-pob_no_AM)/pob_total_nac
per_no_am  = str(round(per_no_am*100,3)) + "%"
Markdown(f"**Porcentaje de habitantes que cuentan con Radio FM a nivel Nacional**: {per_no_fm}")

In [ ]:
Markdown(f"**Porcentaje de habitantes que cuentan con Radio AM a nivel Nacional**: {per_no_am}")

---
## *Fuentes de consulta y referencias*

1. **Sistema de Consulta y Preanálisis de Coberturas de Radiodifusión en línea**, *Instituto Federal de Telecomunicaciones (IFT)*;  http://mapasradiodifusion.ift.org.mx/CPCREL-web/consultaCoberturas/consultaCoberturaspasodos.xhtml?dswid=-2197 
2. **Censo de Población y Vivienda 2010**, *Instituto Nacional de Estadística, Geografía e Información (INEGI)*; https://www.inegi.org.mx/programas/ccpv/2010/
3. **JupyterLab Documentation**, *Project Jupyter.*;https://jupyterlab.readthedocs.io/en/latest/.
4. **Python 3.8 Documentation**, *Python Software Foundation*; https://www.python.org/doc/
5. **Project pip**, *Python Software Foundation*;  https://pypi.org/project/pip/
6. **Repositorio Github by That C# guy**, **


## *Apéndice: Consideraciones técnicas*

1. **Jenks natural breaks:** Se empleó el método de optimización de Jenks; también conocido como *Método de rupuras naturales* para la clasificación de las escalas de colores, con el objetivo de darle claridad a los segmentos de datos en la visualización; este método de clasificación/optimización permite generar una visualización de datos (principalmente empleado en visualización de datos georeferenciados) óptima al ojo humano para observar la diferencia entre las diversas escalas de color empleadas. El algoritmo principal de cálculo consiste en identificar las "rupturas naturales" de un conjunto de datos, de forma que se generan "secciones" o "cubetas" llamados *bins*, de datos, que se consideran representativos de las diferentes escalas de valores para agrupar en un solo color de representación. 

2. 